In [1]:
# Rick Bakker & Amber Stoete, 2018
#
# This Notebook is used for the Knowledge Engineering course at the VU in Amsterdam.
# It is used for the 'monitoring' method of the commonKADS methodology. 
# The system monitors all the medical devices in a surgery room and grades the surgery based on the input from the devices.
# It takes as input some fictional data and grades the fluctuations. 
# To run the system, run every cell, inlcuding this one. Start with the first, than the second, etc. 
# Running the cell is done by Shift + Enter.

# Import requisite libraries
## Imported for displaying grade
import sys
## Imported for the timer
import time

# Initial data
## Monitored value dictionaries
OR0 = {'heart_rate':75,'blood':140}
OR1 = {'heart_rate':80,'blood':144}
OR2 = {'heart_rate':90,'blood':145}
OR3 = {'heart_rate':90,'blood':144}
OR4 = {'heart_rate':90,'blood':140}
OR5 = {'heart_rate':95,'blood':138}
OR6 = {'heart_rate':75,'blood':138}
OR7 = {'heart_rate':75,'blood':140}
OR8 = {'heart_rate':80,'blood':141}
OR9 = {'heart_rate':80,'blood':140}

## List of the monitored value dictionaries
ORS = [OR0, OR1, OR2, OR3, OR4, OR5, OR6, OR7, OR8, OR9]

## Integer length for determining the last run of the system
length = 0

## System model and norm model
System_model = ['heart_rate' , 'blood']
Norms = {'heart_rate':range(70,91), 'blood':range(100,149)}

## Historical data
HD = {'heart_rate': 0, 'blood':0}

## Initial grade
Grade = 10

In [2]:
def F_receive():
    """This function is used for collecting data. It takes every ORx as input, one every second."""
    for OR in ORS:
        # Start F_select with a OR from the ORS list
        F_select(OR)
        
        # Wait one second, before starting F_select with the next OR
        time.sleep(1)

In [3]:
def F_select(OR):
    """Select all the parameters from the system model if they occur in the monitored values and store them in list z."""
    # Make a list with every parameter from 'OR'
    z =  [x for x in System_model if x in OR.keys()] 
    
    # Start F_specify with list 'z' and the 'OR' from the previous function
    F_specify(z, OR)

In [4]:
def F_specify(z, OR):
    """Select all the norms for every item in list z."""
    # Make a list of norms. Take every item in the list 'z' and select the right parameter from the norms model
    norms_list = [Norms[x] for x in z] 
    
    # Start the F_compare function
    F_compare(norms_list, OR)

In [5]:
def F_compare(norms, OR):
    """This function takes every item in the list with norms and compares it with the actual monitored data 
    and calculates the difference."""
    # Difference dictionary
    difference = {}
    

    # Do the if statement as many times as the list 'norms' long is
    for i in range(len(norms)):
        
        # The is statement checks whether the value of the 'OR' is in the range of the norms
        if list(OR.values())[i] not in norms[i]:
            # When true, it calculates the difference between the norm and the received value
            difference[list(OR.keys())[i]] = (min([(list(OR.values())[i] - (min(norms[i]))), (abs(list(OR.values())[i] - (max(norms[i]))))]))
        else:
            # When the received value is in the norms, the difference is 0:
            difference[list(OR.keys())[i]] = 0
            
    # Start the F_classify with the difference and the 'OR'
    F_classify(difference, OR)    

In [6]:
def F_classify(d,OR):
    """The final function classifies whether the difference is a minor or a major disturbance and changes the grade if needed."""
    # Make 'HD', 'Grade', and 'length' global, so they can be accessed within a function
    global HD
    global Grade
    global length
    
    # Initial weight
    Weight = 0
    # List for difference between 'OR' and 'HD'
    new_difference = {}
    
    # Do the if statement as many times as the list 'd', difference, long is
    for i in range(len(d)):
        
        # If there is no historical data, the sum of all the values in this dictionary will be 0
        if sum(list(HD.values())) == 0:
            # If the sum is 0, the difference between the historical data and the received data will be 0
            new_difference[list(d.keys())[i]] = abs(int(list((d.values()))[i]))
        else:    
            # If the sum is unequal to 0, this statement below takes the highest difference between the received value and
            # the historical data
            new_difference[list(d.keys())[i]] = abs(int(list((OR.values()))[i]) - int(list((HD.values()))[i]))   
            
        # If the historical data is 0 and the difference between the historical data and the received value is smaller than 6, 
        # the system does nothing
        if list(d.values())[i] == 0 and list(new_difference.values())[i] < 6 : 
            pass
        
        # Otherwise the system will check whether the disturbance is minor or major
        else: 
            # The disturbance is minor if the difference between the 'OR' and 'norms' and 
            # between 'OR' an d 'HD' is smaller than 6
            
            if list(d.values())[i] < 6 and list(new_difference.values())[i] < 6 :
                # Weight for a minor disturbance
                Weight = 0.2
                
            # If one of the two is larger than 6, the disturbance is major
            else:
                # Weight for a major disturbance
                Weight = 1

            # This statement lowers the grade based on the nature of the disturbance. 
            # In this case, a disturbance of the heart rate is more important than a disturbance in blood pressure.
            if list(d.keys())[i] == 'heart_rate':
                # If the nature of the disturbance is the heart rate:
                Grade -= 0.75*Weight
                
            # With the use of an 'elif' statement, the entire statement can be expanded. 
            # This has to be done when new types of parameters are introduced in the initial data. 
            # The rest of the code will work, but this has to be expanded
            
            elif list(d.keys())[i] == 'blood':
                # If it is the blood pressure:
                Grade -= 0.25*Weight

    # Set the historical data to the received values of 'OR'            
    HD = OR  
    
    # The grade cannot be lower than 1. If it is, set it back to 1
    if Grade < 1:
        Grade = 1
    
    # This bit is for displaying the grade
    ## The length is a integer that is increased by 1, every time this function runs
    length += 1
    
    ## When the length is not equal to the length of 'ORS', the grade is displayed as 'Grade: '
    if length != len(ORS):
        ## Print the grade in a nice format
        sys.stdout.write("\rGrade: {}".format(str(round(Grade,2))+ '   '))
        ## Remove the previous printed grade
        sys.stdout.flush()
    
    ## When the length is equal to the length of 'ORS', the grade is displayed as 'Final grade: '
    ## This happens when the last 'OR' is passed through and the final grade is calculated
    else:
        ## Print the grade in a nice format
        sys.stdout.write("\rFinal grade: {}".format(str(round(Grade,2))+ '   '))
        ## Remove the previous printed grade
        sys.stdout.flush()

In [7]:
# Start the model. The grade is displayed below. 
# ! Do not start the function a second time without running all the cells above a second time !
F_receive()

Final grade: 8.35   